In [3]:
# Imports
from azure.common.credentials import ServicePrincipalCredentials
from azure.identity import ClientSecretCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.storage import StorageManagementClient
from azureml.core import Workspace
# from azure.ai.ml.entities import Workspace
import yaml

In [4]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

subscription_id = config.get("subscription_id")
client_id = config.get("client_id")
secret = config.get("secret")
tenant_id = config.get("tenant_id")


In [5]:
credential = ClientSecretCredential(
    client_id=client_id,
    client_secret=secret,
    tenant_id=tenant_id
)

from azureml.core.authentication import ServicePrincipalAuthentication

credential_aml = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=client_id,
    service_principal_password=secret)

# credential_sp = ServicePrincipalCredentials(
#     client_id=client_id,
#     secret=secret,
#     tenant=tenant_id
# )

In [6]:
# Configuration Parameters
resource_group_name = 'AI_IC_NAM_GenAI-Template-1'
location = 'eastus'
vnet_name = 'MyVNet'
subnet_name = 'MySubnet'
aml_workspace_name = 'genai-test-workspace'
storage_account_name = 'genaitemplate1storage'
openai_account_name = 'genai-test-openai'

resource_client = ResourceManagementClient(credential, subscription_id)
network_client = NetworkManagementClient(credential, subscription_id)
storage_client = StorageManagementClient(credential, subscription_id)

In [7]:
# Create Resource Group
resource_client.resource_groups.create_or_update(resource_group_name, {'location': location})

In [7]:
# Create Virtual Network and Subnet
vnet_params = {
    'location': location,
    'address_space': {
        'address_prefixes': ['10.0.0.0/16']
    }
}
network_client.virtual_networks.begin_create_or_update(resource_group_name, vnet_name, vnet_params)

In [8]:
subnet_params = {
    'address_prefix': '10.0.0.0/24',
    'service_endpoints': [{'service': 'Microsoft.CognitiveServices'}]
}
network_client.subnets.begin_create_or_update(resource_group_name, vnet_name, subnet_name, subnet_params)

In [9]:
# Add Network Rule for Subnet
subnet = network_client.subnets.get(resource_group_name, vnet_name, subnet_name)
subnet_id = subnet.id

In [10]:
# Create Blob Storage
storage_params = {
    'sku': {'name': 'Standard_LRS'},
    'kind': 'StorageV2',
    'location': location,
    'network_acls': {'default_action': 'Deny'}
}
storage_client.storage_accounts.begin_create(resource_group_name, storage_account_name, storage_params)

In [8]:

# Create AML Workspace with VNet
aml_workspace = Workspace.create(
    name=aml_workspace_name,
    auth=credential_aml,
    subscription_id=subscription_id,
    resource_group=resource_group_name,
    location=location,
    create_resource_group=False,
    exist_ok=True,
    friendly_name=aml_workspace_name,
)
